In [1]:
import os
import face_recognition
import numpy as np
from PIL import Image

# สร้างฟังก์ชั่นชื่อ process_faces ที่รับพารามิเตอร์สามตัวคือ data_dir, processed_faces_dir, processed_embeddings_dir
# โดยแต่ละพารามิเตอร์คือ path ของไดเร็กทอรี่ที่แตกต่างกัน
#ระบบจัดเก็บ ไดเร็กเตอร์รี่
#projectfacerecognitionv2/ (ชื่อโปรเจคนี้)
#├── dataset/              เก็บรูปภาพต้นฉบับของแต่ละคน
#│   ├── person_01/ (โฟลเดอร์)
#│   │   ├── image1.jpg (ไฟล์ภาพ)
#│   │   └── image2.jpg
#│   ├── person_02/
#│   │   ├── image1.jpg
#│   │   └── image2.jpg
#│
#├── processed/      (เก็บข้อมูลภาพที่ประมวลผลไปเเล้ว)
#│   ├── faces/        (กรอบใบหน้าจากคำสั่ง face_location)
#│   │   ├── person_01_face1.jpg
#│   │   ├── person_01_face2.jpg
#│   │   ├── person_02_face1.jpg
#│   │   ├── person_02_face2.jpg
#│   │   └── ...
#│   └── embeddings/    (เก็บรหัสใบหน้า)
#│       ├── person_01_face1.npy  (รหัสใบหน้าได้จากคำสั่ง face_encoding)
#│       ├── person_01_face2.npy
#│       ├── person_02_face1.npy
#│       ├── person_02_face2.npy
#│       └── ...
#└── script/               (เก็บสคริปต์สำหรับการประมวลผล)
#    ├── preprocess.ipynb (่jupyter เอาไว้ดูเเละเขียนเป็นเเต่ละขั้นตอนไปตามที่คิดไว้)
#    ├── trainpicture.py (รันโค้ดที่สมบูรณ์เเบบในที่ยังเป็นเเค่ส่วนของการเก็บข้อมูลใบหน้าเเละรหัสไว้เเยกต่างหาก)

def process_faces(data_dir, processed_faces_dir, processed_embeddings_dir):
    #   ---------------------------------------------------------------------
    #   ฟังก์ชันนี้สร้างมาเพื่อการประมวลผลภาพในชุดข้อมูลเพื่อตรวจจับใบหน้าและดึงข้อมูลการเข้ารหัสใบหน้า
    #   จากนั้นบันทึกภาพใบหน้าที่ตรวจจับได้และการเข้ารหัสใบหน้าไปยังไดเร็กทอรี่ที่กำหนด 
    #   พารามิเตอร์:
    #   data_dir (str) path ไปยังไดเร็กทอรี่ที่เก็บชุดข้อมูลของภาพ
    #   processed_faces_dir (str) path ไปยังไดเร็กทอรี่เพื่อบันทึกภาพใบหน้าที่ตัดมาเเล้ว
    #   processed_embeddings_dir (str) path ยังไดเร็กทอรี่เพื่อบันทึกการเข้ารหัสใบหน้า
    #   ----------------------------------------------------------------------
    # ใช้ตัวเเปร person_name ลูปผ่านแต่ละโฟลเดอร์ของบุคคลในชุดข้อมูล dateset จะไล้ตั้งเเต่ person_01 ---> person_26 เลย
    for person_name in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person_name)
        # ตรวจสอบว่า person_dir เป็นไดเร็กทอรี่หรือไม่
    if  os.path.isdir(person_dir): #ถ้าไม่เป็นโค้ดที่เหลิอจะไม่รันเลยเพราะ if จะรันเมื่อเงื่อนไขเป็นจริง

        # สร้างโฟลเดอร์ย่อยสำหรับแต่ละบุคคลใน processed_faces_dir และ processed_embeddings_dir
        person_faces_dir = os.path.join(processed_faces_dir, f"faces_{person_name}")
        person_embeddings_dir = os.path.join(processed_embeddings_dir, f"encodings_{person_name}")
        
        os.makedirs(person_faces_dir, exist_ok=True)
        os.makedirs(person_embeddings_dir, exist_ok=True)
        

        # ลูปผ่านแต่ละภาพในไดเร็กทอรี่ของบุคคล
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            print(f"Processing file: {image_path}")

            
            try:
                # โหลดภาพโดยใช้ face_recognitio
                image = face_recognition.load_image_file(image_path)
                # ตรวจจับตำแหน่งใบหน้าในภาพ ใช้ face_locations
                face_locations = face_recognition.face_locations(image)
                # เปลี่ยนเป็นรหัสภาพที่โหลดเเละตัดให้เหลือเเต่ใบหน้ามา
                encodings = face_recognition.face_encodings(image, face_locations)

                 #ถ้าไม่พบใบหน้าใดๆ หรือก็คือใน path ไฟล์นั้นอาจจะหาใบหน้าไม่เจอ
                if len(face_locations) == 0:
                 print(f"No faces found in {image_path}")
                #ถ้าไม่จริงก็มาเงื่อนไขนี้บอกจำนวนใบหน้าที่เจอ
                else:
                    print(f"Found {len(face_locations)} face(s) in {image_path}")
                # ---------------------------------------------------------
                # ลูปผ่านแต่ละใบหน้าที่ตรวจจับได้และการเข้ารหัสที่สอดคล้องกัน
                # ก็คือตัวเเปร i จะเป็นลำดับ เเละ enumerate กับ zip จะช่วยให้การจับคู่ดีขึ้น
                # สมมุติว่า ถ้าเราใช้ zip อย่างเดียวจะจัดลำดับดังนี้
                #    [(1, 15, 20), [0.1, 0.2, 0.3]] ค่าของลิสเรก คือ (face_locations) เเละค่าลิสสอง คือ (face_encoding)
                #    [(25, 30, 35, ), [0.4, 0.5, 0.6]]
                # เมื่อเราใช้ enumerate กับ zip จะจัดลำดับดังนี้
                #    [0,(1, 15, 20), [0.1, 0.2, 0.3]]
                #    [1,(1, 15, 20), [0.1, 0.2, 0.3]]
                #    .
                #    .
                #    [25,(1, 15, 20), [0.1, 0.2, 0.3]]
                # ---------------------------------------------------------
                for i, (face_location, face_encoding) in enumerate(zip(face_locations, encodings)):
                    top, right, bottom, left = face_location
                    face_image = image[top:bottom, left:right]
                    face_image_pil = Image.fromarray(face_image)
                    face_image_filename = f"{person_name}_face{i+1}.jpg"
                    face_image_path = os.path.join(person_faces_dir, face_image_filename)
                    face_image_pil.save(face_image_path)
                    #เมื่อเราได้ข้อมูลเก็บในส่วนของ ลิส (face_location กับ encodings มาเเล้วเราจะทำการอัปโหลดไฟล์ path ของ
                    #person_embeddings_dir, person_faces_dir ซึ่งสร้างเป็น path เเยกเป็นโฟลเดอร์เเต่ละคนไว้เเล้ว
                    face_encoding_filename = f"{person_name}_face{i+1}.npy"
                    face_encoding_path = os.path.join(person_embeddings_dir, face_encoding_filename)
                    np.save(face_encoding_path, face_encoding)


            except Exception as e:
                     print(f"Error processing file {image_path}: {e}")

In [2]:
# กำหนดไดเร็กทอรี่สำหรับชุดข้อมูลและข้อมูลที่ประมวลผลแล้ว
data_dir = r"C:\Users\class\Desktop\projectfacerecognitionv2\dataset"
processed_faces_dir = r"C:\Users\class\Desktop\projectfacerecognitionv2\processed\faces"
processed_embeddings_dir = r"C:\Users\class\Desktop\projectfacerecognitionv2\processed\embeddings"

os.makedirs(processed_faces_dir, exist_ok=True)
os.makedirs(processed_embeddings_dir, exist_ok=True)

# เรียกฟังก์ชันเพื่อประมวลผลใบหน้า
process_faces(data_dir, processed_faces_dir, processed_embeddings_dir)

Processing file: C:\Users\class\Desktop\projectfacerecognitionv2\dataset\person_9\Thanatcha.jpg
Found 1 face(s) in C:\Users\class\Desktop\projectfacerecognitionv2\dataset\person_9\Thanatcha.jpg
